We now have composably destructible objects. They are automatically
destructed and implicitly handled.
They allow composition and even allow returning from functions.

With that accomplished, we can go ahead and make some cosmetic changes
to our code. After all, this _is_ a C++ conferense, is it not?

One common pain point for C++ programmers writing in Python is `self`.
You have to pass it around _everywhere_.
Wouldn't it be nice if we could write our code like this instead?

In [ ]:
class Reader(CppClass):
    zipfile:ZipFile

    def Reader(path):
        this.zipfile = ZipFile(path)


Well, let's make it happen. We already have most of the building blocks
at hand.

We already decorate all of our methods, and we're used to using global
stacks managed by context managers.
All we need to do is add a stack for `self`, accessed via `this`.

In [ ]:
_this_stack = deque()

class ThisProxy:
    def __getattr__(self, name):
        return getattr(_this_stack[-1], name)

    def __setattr__(self, name, value):
        setattr(_this_stack[-1], name, value)


class ThisScope:
    def __init__(self, instance):
        _this_stack.append(instance)
    def __enter__(self):
        return
    def __exit__(self, exc_type, exc_val, exc_tb):
        _this_stack.pop()


this = ThisProxy()

We did, however, have to proxy access to the stack, as re-assigning
`this` would achieve nothing, as we discussed before.

Another change we'll have to do is decorate methods a bit differently
than we do free functions.
We need to add the `ThisScope` context, and we need to explicitly
handle the `self` argument passed to them.

In [ ]:
def cpp_method(f):
    def _wrapper(self, *args, **kwargs):
        with ThisScope(self):
            with DtorScope():
                retval = f(*args, **kwargs)
                rebind_to_parent_dtor(retval)
                return retval

    return _wrapper

And with that, we can write our `Reader` as follows:

In [ ]:
class Reader(CppClass):
    zipfile:ZipFile

    def Reader(path):
        this.zipfile = ZipFile(path)

    ...